In [41]:
from gensim.models import FastText
import re
import json
import string
from stop_words import get_stop_words
import pickle
import numpy as np
from tqdm import notebook
import json
import pymorphy2
import hnswlib
from string import ascii_letters
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict
import pickle

Для того, чтобы определить наиболее близкий вопрос к заданному полезны могут быть слова, написанные кириллицей и латиницей. Пунктуация в данном датасете обширно используется дя придания экспрессивной окраски, но на содержательную составляющую влияет слабо. Числа могли бы быть полезны, но получаемые результаты не оправдывают их использование.

В некоторых текстах попадаются html теги, которые целесообразно удалить (тематика вопросов всё-таки не такая узкая).

In [2]:
tag_re = re.compile(r'<[^>]+>')

In [3]:
token_re = re.compile(rf'[А-Яа-яЁёA-Za-z]+')

In [4]:
def tokenize(text):
    return re.findall(token_re, text)

In [5]:
def clean(text):
    text = tag_re.sub('', text)
    tokens = tokenize(text)
    return [token.lower() for token in tokens]

In [6]:
EMBEDDING_SIZE = 100
MAX_LINES = 1_000_000

Разделим вопросы и ответы при чтении корпуса.

In [7]:
def read_corpus(path, limit=0):
    questions = []
    answers = defaultdict(lambda: [])
    count = 0
    question_index = -1
    new_question = False
    end = False
    with open(path, 'r') as file:
        for line in notebook.tqdm(file):
            
            count += 1
            
            if line == '\n':
                continue
                
            if limit and count > limit:
                end = True
            

            if line == '---\n':
                new_question = True
                if end:
                    break
            elif new_question:
                questions.append(clean(line))
                new_question = False
                question_index += 1
            else:
                answers[question_index].append(line)
                
    return questions, answers

In [8]:
def preprocess(text):
    text = tag_re.sub('', text)
    tokens = tokenize(text)
    return [token for token in tokens if token not in stop_words]

In [9]:
morph = pymorphy2.MorphAnalyzer()

In [10]:
def normalize(tokens):
    return [morph.parse(token.lower())[0].normal_form for token in tokens]

Дополним стоп-слова глаголами, которые для данной задачи не очень помогают в определении содержания вопросов.

In [11]:
stop_words = set(get_stop_words("ru")).union(set(('стать', 'иметь', 'быть', 'являться')))

def remove_stop_words(tokens):
    return [token for token in tokens if token not in stop_words]

In [12]:
questions, answers = read_corpus('/mnt/f/data/answers.txt', MAX_LINES)

Некоторые вопросы просто огромные. Есть смысл их обрезать.

In [14]:
max([len(tokens) for tokens in questions])

904

In [15]:
np.mean([len(tokens) for tokens in questions])

19.58840011388932

In [17]:
np.median([len(tokens) for tokens in questions])

11.0

In [24]:
questions = [tokens[:min(50, len(tokens) - 1)] for tokens in questions]

В данной концепции есть смысл обучать векторизатор только на вопросах.

In [25]:
questions_norm = []
for sentence in notebook.tqdm(questions):
    questions_norm.append(normalize(sentence))

In [26]:
%%time
phrases = Phrases(questions_norm,
                  common_terms=list(stop_words),
                  threshold=10,
                  min_count=10)
bigram = Phraser(phrases)
questions_norm_ph = list(bigram[questions_norm])

CPU times: user 13.8 s, sys: 859 ms, total: 14.7 s
Wall time: 14.7 s


In [34]:
questions_norm_ph = [remove_stop_words(sent) for sent in questions_norm_ph]

Дополнительно взвесим вектора.

In [35]:
%%time
ft = FastText(sentences=questions_norm_ph,
              size=EMBEDDING_SIZE,
              min_count=20,
              window=3,
              workers=-1,
              max_vocab_size=30_000,
              negative=10,
              bucket=1000
             )

CPU times: user 2.8 s, sys: 125 ms, total: 2.92 s
Wall time: 2.96 s


In [69]:
ft.save('/mnt/f/data/bot/ft.model')

In [36]:
questions_norm_ph_str = [' '.join(tokens) for tokens in questions_norm_ph]

In [37]:
tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(questions_norm_ph_str)
tfidf_feat = tf_idf_vect.get_feature_names()

In [70]:
with open('/mnt/f/data/bot/tfidf.pkl', 'wb') as file:
    pickle.dump(tf_idf_vect, file)

In [38]:
def vectorize_sent(tokens, sentence_num, model):
    vector = np.zeros(model.vector_size)
    n_tokens = len(tokens)
    weight_sum = 0
    
    if not n_tokens:
        return vector
    
    for token in tokens:
        try:
            weight = final_tf_idf[sentence_num, tfidf_feat.index(token)]
        except:
            weight = 0
        vector += (model.wv.get_vector(token) * weight)
        weight_sum += weight
        
    if not weight_sum:
        return vector * 0
        
    vector /= weight_sum
    
    return vector

In [39]:
len(questions_norm_ph)

154536

In [40]:
questions_vect = []
for num, sentence in notebook.tqdm(enumerate(questions_norm_ph)):
    questions_vect.append(vectorize_sent(sentence, num, ft))
                          
questions_vect = np.vstack(questions_vect)

In [42]:
p = hnswlib.Index(space ='cosine', dim=EMBEDDING_SIZE)

In [43]:
p.init_index(max_elements=len(questions_norm_ph), ef_construction=200, M=16)

In [44]:
p.add_items(questions_vect)

In [71]:
p.save_index('/mnt/f/data/bot/index.bin')

In [62]:
with open('/mnt/f/data/bot/answers.json', 'w') as file:
    json.dump(dict(answers), file)

In [50]:
labels, distances = p.knn_query(questions_vect[300], k=3)

In [51]:
labels

array([[  300, 72121, 67385]], dtype=uint64)

In [55]:
questions_norm_ph[300]

['тухлый', 'аж', 'задать_вопрос']

In [54]:
answers[72121]

['Конечно!!! На панели ?. \n',
 'нет..тебя не хватит.а хотя может ты такая влюбвиобильная. \n',
 'Я не думаю что ты влюбилась в них троих, просто ты испытуеш к ним большую симпатию. Определись кто тебе больше нравится, знаешь одной попой на 2 (в твоём случае 3) стула не сядишь.. \n',
 'Устрой одновременно свидание со всеми троими и посмотри, что из этого выйдет:)<br>А зачем ты всех зомбируешь своей рассылкой? Сначала идет игра, а в конце угроза: не отправишь - 10 лет мучаться будешь... Ты сама-то в эту хрень веришь?. \n']